**7.5 부스팅**
- 부스팅: 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
- 아이디어 -> 앞이 모델을 보완해 나가면서 일련의 예측기를 학습시키는 것
- 부스팅 종류
  - 에이다부스트
  - 그레이디언트 부스팅

7.5.1 에이다부스트
- 이전 모델이 과소적합했던 훈련 샘풀의 가중치를 더욱 높여 새로운 예측기가 학습하기 어려운 샘플에 맞춰지도록 하는 기법
- 에이다부스트 분류기 원리
  - 알고리즘이 기반이 되는 첫 번째 분류기를 훈련 세트에서 훈련시키고 예측을 형성함
  - 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높인 후, 두번째 분류기에 업데이트된 가중치를 사용해 훈련 후 예측을 형성
  - 일련의 과정을 반복함
- 이러한 과정은 경사 하강법과 비슷하나, 에이다부스트는 점차 성능이 좋아지도록 앙상블에 예측기를 추가하는 방법임
- 모든 예측기가 훈련을 마치면 이 앙상블은 배깅이나 페이스팅과 같은 방식으로 예측을 형성함 (이때 가중치는 정확도에 따라 다르게 부여됨)
- 가중치는 $\alpha_{i}$로 계산됨
  - 예측기가 정확할수록 가중치는 더욱 높아지게 되고, 무작위이면 0, 나쁘면 음수에 가까워짐
- 사이킷런에선 이러한 에이다부스트의 다중 클래스 버전을 SAMME를 이용해서 사용함
  - 만일 예측기가 클래스의 확률을 추정할 수 있다면 SAMME.R을 이용

In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

# 5장에서의 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [2]:
#AdaBoostClassifier을 이용한 에이다부스트 분류기의 훈련
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

7.5.2 그레이디언트 부스팅
- 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가하는 방법
- 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 것이 아닌, 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시키는 방법을 사용함


In [3]:
# DecisionTreeRegressor를 훈련세트에 학습시키기
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [4]:
# 두번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델을 훈련
y3 = y2-tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=3)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=3)

In [5]:
# y_pred = sum(tree.predict(X_new) for tree in (tee_reg1, tree_reg2, tree_reg3))

In [6]:
# 사이킷런의 GradientBoostingRegressor을 이용한 GBRT 앙상블 훈련
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

- learning_rate: 각 트리의 기여정도를 조절
  - learning_rate를 낮게 설정하면 앙상블을 훈련 세트에학습시키기위해 많은 트리가 필요하나, 예측 성능은 좋아짐 (축소)

In [7]:
# 최적의 트리수를 찾기위한 조기종료 수행
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
 for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=np.int64(120))

In [8]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
errir_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error= mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up  +=1
    if error_going_up == 5:
      break # 조기 종료

- warn_start = True : 사이킷런의 fit() 메서드가 호출될 때 기존 트리를 유지하면서 훈련을 추가할 수 있도록 함
- GradientBoostingRegressor: 확률적 그래디언트 부스팅으로 사용할 수 있음
  - subsample: 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있음
  - 대표적으로 XGBoost를 이용해 최적화된 그레이디언트 부스팅을 이용함
  - XGBoost는 빠른 속도와 확장성, 그리고 이식성이 있어 많이 이용됨  


In [14]:
from xgboost import XGBRegressor
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [12]:
xgb_reg = xgboost.XGBRegressor(early_stopping_rounds=2)
xgb_reg.fit(X_train, y_train,eval_set = [(X_val, y_val)])
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.38724
[1]	validation_0-rmse:0.30876
[2]	validation_0-rmse:0.25683
[3]	validation_0-rmse:0.22513
[4]	validation_0-rmse:0.20948
[5]	validation_0-rmse:0.19326
[6]	validation_0-rmse:0.17987
[7]	validation_0-rmse:0.17562
[8]	validation_0-rmse:0.16917
[9]	validation_0-rmse:0.16445
[10]	validation_0-rmse:0.16076
[11]	validation_0-rmse:0.15821
[12]	validation_0-rmse:0.15652
[13]	validation_0-rmse:0.15536
[14]	validation_0-rmse:0.15397
[15]	validation_0-rmse:0.15291
[16]	validation_0-rmse:0.15263
[17]	validation_0-rmse:0.15240
[18]	validation_0-rmse:0.15185
[19]	validation_0-rmse:0.15178
[20]	validation_0-rmse:0.15169
[21]	validation_0-rmse:0.15166
[22]	validation_0-rmse:0.15161
[23]	validation_0-rmse:0.15159
[24]	validation_0-rmse:0.15158
[25]	validation_0-rmse:0.15156
[26]	validation_0-rmse:0.15155
[27]	validation_0-rmse:0.15144
[28]	validation_0-rmse:0.15143
[29]	validation_0-rmse:0.15142
[30]	validation_0-rmse:0.15141
[31]	validation_0-rmse:0.15136
[32]	validation_0-

**7.6 스태킹**
- 앙상블에 속한 모든 예측기의 예측을 취합하는 모델을 훈련시키는 바업ㅂ
- 각 예측기에서 각각의 훈련 세트를 예측하고, 이에 대한 예측 결과를 마지막 예측기(블렌더, 메타 학습기)에서 입력으로 받아 최종예측을 수행
  - 대부분 블렌더를 학습시키는 방법은 홀드 아웃 세트를 사용
- 스테킹의 방법
  - 훈련 세트를 두개의 서브셋으로 나누어 하나의 서브셋을 첫번째 레이어의 예측을 훈련하는데 사용함
  - 첫번째 예측기를 이용해 두 번째 세트에 대한 예측을 만들고, 각 예측은 모두 서로에게 새로운 값이 형성됨
  - 따라서 각 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들어 이를 블렌더에 훈련 시킴
- 스태킹은 블렌더를 여러개 훈련 시키는 것도 가능
  - 블렌더의 레이어가 형성 됨
- 스태킹은 아직 사이킷런에서 지원되지 않음